In [1]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import keras_tuner as kt

In [2]:
# prepare the dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# since we need a validation set
# we use 80% of the training set for training, 20% for validation purpose
pivot_index = round(len(X_train)*0.8)
# since the data is in 2d numpy array
# change it to 1d numpy array
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_val = X_train[pivot_index:]
X_train = X_train[:pivot_index]
y_val = y_train[pivot_index:]
y_train = y_train[:pivot_index]

In [13]:
# build the CNN model
model = keras.models.Sequential([
    keras.layers.Conv2D(8, 5, activation='relu', padding='same', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(16, 2, activation='relu', padding='same'),
    keras.layers.Conv2D(16, 2, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(32, 2, activation='relu', padding='same'),
    keras.layers.Conv2D(32, 2, activation='relu', padding='same'),
    keras.layers.Conv2D(64, 2, activation='relu', padding='same'),
    keras.layers.Conv2D(64, 2, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 2, activation='relu', padding='same'),
    keras.layers.Conv2D(128, 2, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.save('my_first_cnn.h5')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 28, 28, 8)         208       
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 16)        528       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 28, 28, 16)        1040      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 14, 14, 32)        2080      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 14, 14, 32)        4128      
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 14, 14, 64)       

In [14]:
# set up a checkpoints and early stopping condition
checkpoint = keras.callbacks.ModelCheckpoint('my_first_cnn.h5', save_best_only=True)
earlystop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[checkpoint, earlystop])

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 35s 724us/sample - loss: 0.7956 - acc: 0.7194 - val_loss: 0.1401 - val_acc: 0.9654
Epoch 2/100
48000/48000 [==============================] - 34s 705us/sample - loss: 0.2938 - acc: 0.9142 - val_loss: 0.0904 - val_acc: 0.9794
Epoch 3/100
48000/48000 [==============================] - 34s 703us/sample - loss: 0.2165 - acc: 0.9393 - val_loss: 0.0914 - val_acc: 0.9770
Epoch 4/100
48000/48000 [==============================] - 34s 703us/sample - loss: 0.1656 - acc: 0.9567 - val_loss: 0.0728 - val_acc: 0.9799
Epoch 5/100
48000/48000 [==============================] - 34s 706us/sample - loss: 0.1305 - acc: 0.9656 - val_loss: 0.0503 - val_acc: 0.9868
Epoch 6/100
48000/48000 [==============================] - 34s 708us/sample - loss: 0.1176 - acc: 0.9689 - val_loss: 0.0446 - val_acc: 0.9885
Epoch 7/100
48000/48000 [==============================] - 34s 701us/sample - loss: 0.1072 - acc: 

In [15]:
# get the model performance in test data
model.evaluate(X_test, y_test)

10000/10000 [==============================] - 2s 220us/sample - loss: 0.0405 - acc: 0.9898


[0.040529092000618445, 0.9898]